In [ ]:
nz = 100
import torch.nn as nn
import torch.nn.functional as F
#wgan generator
class wgan_generator(nn.Module):
    def __init__(self):
        super(wgan_generator,self).__init__()
        #input: [batch_size,100]
        self.fc1 = nn.utils.weight_norm(nn.Linear(nz,2*4*4*1024))
        
        self.conv1 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(1024,2*512,kernel_size=5,padding=2,stride=1))
        )
        self.conv2 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(512,2*256,kernel_size=5,padding=2,stride=1))
        )
        
        self.conv3 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(256,2*128,kernel_size=5,padding=2,stride=1))
        )
        
        self.conv4 = nn.utils.weight_norm(nn.Conv2d(128,3,kernel_size=5,padding=2,stride=1))
        


    def forward(self,x):
        
        x = self.fc1(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l) # gated linear unit, one of Alec's tricks
        
        x = x.view(-1,1024,4,4)
        #[4,4]
        x = self.conv1(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[8,8]
        x = self.conv2(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[16,16]
        x = self.conv3(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[32,32]
        x = self.conv4(x)
        
        x = self.tanh(x)
        return x;


# 测试cuda代码

In [ ]:
    import numpy as np
    import ctypes
    ll = ctypes.cdll.LoadLibrary
    lib = ll("./libhungarian.so")
    #lib.func.argtypes = [ctypes.c_float]
    #set up function
    c = np.abs(np.random.randn(600,600))
    simplex_func = lib.func
    simplex_func.restype = ctypes.POINTER(ctypes.c_int)
    simplex_func.argtypes = [ctypes.POINTER(ctypes.c_float)]
    #print simplex_func()
    #print '***finish***'
    #batch_size = 10
    #build simplexTableau
    c =  c.flatten().tolist()
    c_value =(ctypes.c_float * len(c))(*c)
    result = simplex_func(c_value)
    for i in range(batch_size):
    	pi[result[i]] = 1.0/batch_size
    pi = pi.reshape(batch_size,-1)
    return pi

In [1]:
import numpy as np
import ctypes
ll = ctypes.cdll.LoadLibrary
lib = ll("./libhungarian.so")
#lib.func.argtypes = [ctypes.c_float]
#set up function
simplex_func = lib.func


AttributeError: ./libhungarian.so: undefined symbol: func

In [ ]:
import numpy as np
c = np.abs(np.random.randn(600,600))
print c

In [ ]:
a = torch.randn(4,5,80)

b = torch.chunk(a,20,dim=2)

print b[0].shape
print a.shape

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torch.optim as optim
import torchvision.utils as vutils
import numpy as np
from scipy.optimize import linprog
import torch.nn.functional as F
import utils

g_steps = 5
result_directory = "cifar_real_samples"
batch_size = 64
bn = True
lr = 3e-4
beta = (0.5,0.999)
epoch_num = 200
cuda = True

#Linear Program
a1 = np.zeros([batch_size*batch_size,batch_size]);
a2 = np.zeros([batch_size*batch_size,batch_size])
for i in range(batch_size):
    a1[batch_size*i:batch_size*(i+1),i] = 1
    a2[batch_size*i:batch_size*(i+1),:] = np.eye(batch_size,batch_size)
A = np.concatenate((a1,a2),axis=1)
A = A.T
b = np.ones([batch_size*2])/batch_size

#wgan generator
class wgan_generator(nn.Module):
    def __init__(self):
        super(wgan_generator,self).__init__()
        #input: [batch_size,100]



    def forward(self,x):


        return x;

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

def loss_fn(real_x,fake_x,isopt=True,pi=None):
    batch_size = real_x.size(0)
    cost_matrix = torch.zeros([batch_size,batch_size]).cuda()
    dist_matrix = np.zeros([batch_size,batch_size])
    for i in range(batch_size):
        diff = real_x - fake_x[i]
        diff = diff.view(diff.size(0),-1)
        #||x_i - y_j||_2
        #[batch_size,1]
        diff_norm = diff.norm(2 , dim = 1)
        #||x_i - y_j||_1
        diff_norm_1 = diff.norm(1 , dim = 1)
        #cosin(x_i,y_j)
        
        #sobolev spaces
        
        cost_matrix[:,i] = diff_norm + 0.05 * diff_norm_1
        #min_value,min_pos = torch.min(diff_norm,dim=0)
        #print min_pos
        #dist_matrix[min_pos,i] = 1.0/batch_size
    #[batch_size,batch_size]
    if isopt:
        c = cost_matrix.cpu().detach().numpy().reshape(batch_size*batch_size)
        pi = utils.simplexCVX(batch_size,c);
        pi = torch.from_numpy(pi).float().cuda()
        #print pi[0,:]
    #assert(type(pi)!=Nonetype)
    loss = torch.sum(torch.mul(cost_matrix,pi))
    return loss,pi

def train(g_net):
    data = dset.CIFAR10(root="/home/lrh/dataset/cifar-10",train = False, download=True,transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ]))

    dataloader = torch.utils.data.DataLoader(data,batch_size=batch_size,shuffle=True,drop_last=True)
    g_optimizer = optim.Adam(g_net.parameters(),lr=lr,betas = beta)

    for epoch in range(epoch_num):
        for i,data in enumerate(dataloader,0):
        #     #with shape [batch_size,3,32,32]
             real_x = data[0]
        #     z = torch.randn(real_x.size(0),100);
        #     #to GPU
        #     if cuda:
        #         real_x = real_x.cuda()
        #         z = z.cuda()
        #
        #     g_optimizer.zero_grad()
        #
        #     isopt = True
        #     pi = None
        #     for j in range(g_steps):
        #         fake_x = g_net(z)
        #         #[batch_size,3,32,32]
        #         loss,pi = loss_fn(real_x,fake_x,isopt,pi)
        #         loss.backward()
        #         g_optimizer.step()
        #         isopt = False
        #
        #
        #     print "epoch is:[{}|{}],index is:[{}|{}],g_loss:{}".\
        #         format(epoch,epoch_num,\
        #         i,len(dataloader),loss);
        # fake_x = g_net(z)
        vutils.save_image(real_x.cpu().detach(),'%s/real_samples_epoch_%03d.png'
        % (result_directory,epoch),normalize=True)
        if epoch%50==0:
            torch.save(g_net.state_dict(),'%s/gnet_%03d.pkl' %(result_directory,epoch));


if __name__=="__main__":

    g_net = wgan_generator()
    g_net.apply(weights_init)
    if cuda:
        g_net.cuda()
    #set moudle.istraing=True
    #net.train()
    train(g_net)


# one_batch samples


get a small numbers of samples




**探究inception score的数学特性**

batch_size:5000 

9.85611644126555+0.503533570304311

batch_size:1000

8.291916482024153+0.43865066427286864

batch_size:64

3.4658015717169794+0.5964037687766206

batch_size:100

4.437662900437032+0.7272216898071227

batch_size:200

5.6565756770326, 0.645820972999001

batch_size:400

6.790523369170593, 0.7863464677535649

batch_size:600

7.517676544251043, 0.688784609754242

$$inception\_score = e^{E(\sum_ip(y_i|x)log(\frac{p(y_i|x)}{p(y_i)}))}$$


In [8]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy

def inception_score(imgs, cuda=True, batch_size=32, resize=False, splits=1):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    cuda -- whether or not to run on GPU
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """
    N = len(imgs)

    assert batch_size > 0
    assert N > batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)


class cifarDataset(torch.utils.data.Dataset):
        def __init__(self,images):
            self.data = images
        def __getitem__(self,index):
            return self.data[index]
        def __len__(self):
            return len(self.data)

In [37]:
if __name__ == '__main__':
    
    class cifarDataset(torch.utils.data.Dataset):
        def __init__(self,images):
            self.data = images
        def __getitem__(self,index):
            return self.data[index]
        def __len__(self):
            return len(self.data)

    import torchvision.datasets as dset
    import torchvision.transforms as transforms

    cifar = dset.CIFAR10(root='/home/lrh/dataset/cifar-10', download=False,
                             transform=transforms.Compose([
                                 transforms.Scale(32),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                             ])
    )
    
    dataloader = torch.utils.data.DataLoader(cifar,batch_size=64,shuffle=True,drop_last=True)
    for i,data in enumerate(dataloader):
        print data[0].shape
        cifar = cifarDataset(data[0])
        break;
        
    #IgnoreLabelDataset(cifar)
    print ("Calculating Inception Score...")
    print (inception_score(cifar, cuda=True, batch_size=32, resize=True, splits=10))


torch.Size([64, 3, 32, 32])
Calculating Inception Score...
(3.6043417909946895, 0.5218633565924065)


/home/lrh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


# 计算训练好的模型的inception score

c:2_norm+0.05*1_norm

IS-64:2.9020586246659446, 0.5355145676387948

IS-100:3.575182050796198, 0.5209824971289462

IS-200:4.38855363712337, 0.5608734183151969

IS-400:4.8389852230529815, 0.560834277123877

IS-600:5.5008282662264305, 0.389602242839442

In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
nz = 100

class wgan_generator(nn.Module):
    def __init__(self):
        super(wgan_generator,self).__init__()
        #input: [batch_size,100]
        self.fc1 = nn.utils.weight_norm(nn.Linear(nz,2*4*4*1024))

        self.conv1 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(1024,2*512,kernel_size=5,padding=2,stride=1))
        )

        self.conv2 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(512,2*256,kernel_size=5,padding=2,stride=1))
        )

        self.conv3 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(256,2*128,kernel_size=5,padding=2,stride=1))
        )

        self.conv4 = nn.utils.weight_norm(nn.Conv2d(128,3,kernel_size=5,padding=2,stride=1))



    def forward(self,x):

        x = self.fc1(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l) # gated linear unit, one of Alec's tricks

        x = x.view(-1,1024,4,4)
        #[4,4]
        x = self.conv1(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[8,8]
        x = self.conv2(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[16,16]
        x = self.conv3(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[32,32]
        x = self.conv4(x)

        x = F.tanh(x)
        return x;
    
    
load_gnet_directory = "/home/lrh/program/git/pytorch-example/p-gan/result_smallcifar_1017_direct_all600data_onebatch_0.05LAMBDA/gnet_52000.pkl"
#load_gnet_directory = "/home/lrh/program/git/pytorch-example/p-gan/result_smallcifar_1015_direct_64data_onebatch_0.05LAMBDA/gnet_60000.pkl"
g_net = wgan_generator()
g_net.cuda()
g_net.load_state_dict(torch.load(load_gnet_directory))

z = torch.randn(600,nz);
z = z.cuda()
images = g_net(z)
dataset = cifarDataset(images)
print inception_score(dataset,resize=True,splits=10)
print (inception_score(cifar, cuda=True, batch_size=32, resize=True, splits=10))

/home/lrh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


(5.5008282662264305, 0.389602242839442)
(3.6043417909946895, 0.5218633565924065)


**真实图片与噪声的二阶范数**

直接在生成图片的空间上做：

1.探究高维数据n维范数失效的问题

2.cos距离和其他的度量空间

3.衡量图片的相似性指标

    -ssim
    
    -msssim


    

In [ ]:

import torch
import torchvision.datasets as dset
import torchvision.transforms as transforms
import utils
import numpy as np
batch_size = 64

a1 = np.zeros([batch_size*batch_size,batch_size]);
a2 = np.zeros([batch_size*batch_size,batch_size])
for i in range(batch_size):
    a1[batch_size*i:batch_size*(i+1),i] = 1
    a2[batch_size*i:batch_size*(i+1),:] = np.eye(batch_size,batch_size)
A = np.concatenate((a1,a2),axis=1)
A = A.T
b = np.ones([batch_size*2])/batch_size

def loss_fn(real_x,fake_x,isopt=True,pi=None):
    batch_size = real_x.size(0)
    real_x = real_x.view(batch_size,-1)
    fake_x = fake_x.view(batch_size,-1)
    
    cost_matrix = torch.zeros([batch_size,batch_size]).cuda()
    dist_matrix = np.zeros([batch_size,batch_size])
    
    
    for i in range(batch_size):
        diff = real_x - fake_x[i]
        #||x_i - y_j||_2
        #[batch_size,1]
        diff_norm = diff.norm(2 , dim = 1)
        #||x_i - y_j||_1
        diff_norm_1 = diff.norm(1 , dim = 1)
        #cost_matrix[:,i] = diff_norm
        #+ 0.05 * diff_norm_1
        #min_value,min_pos = torch.min(diff_norm,dim=0)
        #print min_pos
        #dist_matrix[min_pos,i] = 1.0/batch_size
        #cost_matrix[i,i] = 9999
    #[batch_size,batch_size]
    #cosin distance
    cos_diff = cosin_loss(real_x,fake_x)
    cost_matrix = cos_diff
    
    if isopt:
        c = cost_matrix.cpu().detach().numpy().reshape(batch_size*batch_size)
        pi = utils.simplexCVX(batch_size,c);
        pi = torch.from_numpy(pi).float().cuda()
        #print pi[0,:]
    #assert(type(pi)!=Nonetype)
    loss = torch.sum(torch.mul(cost_matrix,pi))
    return loss,pi

def cosin_loss(real_x,fake_x):
    #cosin distance
    norm_real_x = real_x / real_x.norm(2,dim=1,keepdim=True)
    norm_fake_x = fake_x / fake_x.norm(2,dim=1,keepdim=True)
    
    #shape [batch_size,batch_size]
    inner_product = torch.matmul(norm_real_x,norm_fake_x.transpose(1,0))
    loss = 1 - inner_product
    return loss

data = dset.CIFAR10(root="/home/lrh/dataset/cifar-10",train = False, download=True,transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ]))


dataloader = torch.utils.data.DataLoader(data,batch_size=batch_size,shuffle=True,drop_last=True)

for i,data in enumerate(dataloader):
    if i==0:
        img1 = data[0].cuda()
    elif i==1:
        img2 = data[0].cuda()
    else:
        break

print loss_fn(img1,img2)

g = (torch.randn(batch_size,3,32,32)-0.5).cuda()
    

print loss_fn(img2,g)  


img1 = img1.view(batch_size,-1)
img2 = img2.view(batch_size,-1)
g = g.view(batch_size,-1)

#print (img1-img2).norm(2,dim=1).mean()
#print (img1-g).norm(2,dim=1).mean()

#print cosin_loss(img1,img2)


## cifar生成


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torch.optim as optim
import torchvision.utils as vutils
import numpy as np
from scipy.optimize import linprog
import torch.nn.functional as F
import utils

nz = 100
g_steps = 5
result_directory = "result_cifar_1009_direct_onebatch16_0.05LAMBDA"
batch_size = 16
bn = True
lr = 3e-4
beta = (0.5,0.999)
epoch_num = 1000
cuda = True


a1 = np.zeros([batch_size*batch_size,batch_size]);
a2 = np.zeros([batch_size*batch_size,batch_size])
for i in range(batch_size):
    a1[batch_size*i:batch_size*(i+1),i] = 1
    a2[batch_size*i:batch_size*(i+1),:] = np.eye(batch_size,batch_size)
A = np.concatenate((a1,a2),axis=1)
A = A.T
b = np.ones([batch_size*2])/batch_size


class wgan_generator(nn.Module):
    def __init__(self):
        super(wgan_generator,self).__init__()
        #input: [batch_size,100]
        self.fc1 = nn.utils.weight_norm(nn.Linear(nz,2*4*4*1024))

        self.conv1 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(1024,2*512,kernel_size=5,padding=2,stride=1))
        )

        self.conv2 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(512,2*256,kernel_size=5,padding=2,stride=1))
        )

        self.conv3 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(256,2*128,kernel_size=5,padding=2,stride=1))
        )

        self.conv4 = nn.utils.weight_norm(nn.Conv2d(128,3,kernel_size=5,padding=2,stride=1))



    def forward(self,x):

        x = self.fc1(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l) # gated linear unit, one of Alec's tricks
        x = x.view(-1,1024,4,4)
        #[4,4]
        x = self.conv1(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[8,8]
        x = self.conv2(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[16,16]
        x = self.conv3(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[32,32]
        x = self.conv4(x)

        x = F.tanh(x)
        return x;

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

def loss_fn(real_x,fake_x,isopt=True,pi=None):
    batch_size = real_x.size(0)
    cost_matrix = torch.zeros([batch_size,batch_size]).cuda()
    dist_matrix = np.zeros([batch_size,batch_size])
    for i in range(batch_size):
        diff = real_x - fake_x[i]
        diff = diff.view(diff.size(0),-1)
        #||x_i - y_j||_2
        #[batch_size,1]
        diff_norm = diff.norm(2 , dim = 1)
        diff_norm_1 = diff.norm(1 , dim = 1)
        cost_matrix[:,i] = diff_norm + 0.1 * diff_norm_1
        #min_value,min_pos = torch.min(diff_norm,dim=0)
        #print min_pos
        #dist_matrix[min_pos,i] = 1.0/batch_size
    #[batch_size,batch_size]
    if isopt:
        c = cost_matrix.cpu().detach().numpy().reshape(batch_size*batch_size)
        pi = utils.simplexCVX(batch_size,c);
        pi = torch.from_numpy(pi).float().cuda()
        #print pi[0,:]
    #assert(type(pi)!=Nonetype)
    loss = torch.sum(torch.mul(cost_matrix,pi))
    return loss,pi

def train(g_net):
    data = dset.CIFAR10(root="/home/lrh/dataset/cifar-10",train = False, download=True,transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ]))

    dataloader = torch.utils.data.DataLoader(data,batch_size=batch_size,shuffle=True,drop_last=True)
    g_optimizer = optim.Adam(g_net.parameters(),lr=lr,betas = beta)

    for i,data in enumerate(dataloader,0):
        #     #with shape [batch_size,3,32,32]
        real_x = data[0]
    for epoch in range(epoch_num):
        z = torch.randn(real_x.size(0),nz);
        #to GPU
        if cuda:
            real_x = real_x.cuda()
            z = z.cuda()

        g_optimizer.zero_grad()
        isopt = True
        pi = None
        for j in range(g_steps):
            fake_x = g_net(z)
            #[batch_size,3,32,32]
            loss,pi = loss_fn(real_x,fake_x,isopt,pi)
            loss.backward()
            g_optimizer.step()
            isopt = False


        fake_x = g_net(z)
        vutils.save_image(fake_x.cpu().detach(),'%s/real_samples_epoch_%03d.png'
        % (result_directory,epoch),normalize=True)
        if epoch%50==0:
            torch.save(g_net.state_dict(),'%s/gnet_%03d.pkl' %(result_directory,epoch));
            print "epoch is:[{}|{}],index is:[{}|{}],g_loss:{}".\
                format(epoch,epoch_num,\
                i,len(dataloader),loss);

if __name__=="__main__":

    g_net = wgan_generator()
    g_net.apply(weights_init)
    if cuda:
        g_net.cuda()
    #set moudle.istraing=True
    #net.train()
    train(g_net)


## lsun生成

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torch.optim as optim
import torchvision.utils as vutils
import numpy as np
from scipy.optimize import linprog
import torch.nn.functional as F
import utils

nz = 100
g_steps = 5
result_directory = "result_lsun_1011_direct_onebatch"
batch_size = 16
bn = True
lr = 3e-4
beta = (0.5,0.999)
epoch_num = 1000
cuda = True


a1 = np.zeros([batch_size*batch_size,batch_size]);
a2 = np.zeros([batch_size*batch_size,batch_size])
for i in range(batch_size):
    a1[batch_size*i:batch_size*(i+1),i] = 1
    a2[batch_size*i:batch_size*(i+1),:] = np.eye(batch_size,batch_size)
A = np.concatenate((a1,a2),axis=1)
A = A.T
b = np.ones([batch_size*2])/batch_size


class wgan_generator(nn.Module):
    def __init__(self):
        super(wgan_generator,self).__init__()
        #input: [batch_size,100]
        self.fc1 = nn.utils.weight_norm(nn.Linear(nz,2*4*4*1024))

        self.conv1 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(1024,2*512,kernel_size=5,padding=2,stride=1))
        )

        self.conv2 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(512,2*256,kernel_size=5,padding=2,stride=1))
        )

        self.conv3 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(256,2*128,kernel_size=5,padding=2,stride=1))
        )

        self.conv4 = nn.utils.weight_norm(nn.Conv2d(128,3,kernel_size=5,padding=2,stride=1))



    def forward(self,x):

        x = self.fc1(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l) # gated linear unit, one of Alec's tricks
        x = x.view(-1,1024,4,4)
        #[4,4]
        x = self.conv1(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[8,8]
        x = self.conv2(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[16,16]
        x = self.conv3(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[32,32]
        x = self.conv4(x)

        x = F.tanh(x)
        return x;

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

def loss_fn(real_x,fake_x,isopt=True,pi=None):
    batch_size = real_x.size(0)
    cost_matrix = torch.zeros([batch_size,batch_size]).cuda()
    dist_matrix = np.zeros([batch_size,batch_size])
    for i in range(batch_size):
        diff = real_x - fake_x[i]
        diff = diff.view(diff.size(0),-1)
        #||x_i - y_j||_2
        #[batch_size,1]
        diff_norm = diff.norm(2 , dim = 1)
        diff_norm_1 = diff.norm(1 , dim = 1)
        cost_matrix[:,i] = diff_norm + 0.1 * diff_norm_1
        #min_value,min_pos = torch.min(diff_norm,dim=0)
        #print min_pos
        #dist_matrix[min_pos,i] = 1.0/batch_size
    #[batch_size,batch_size]
    if isopt:
        c = cost_matrix.cpu().detach().numpy().reshape(batch_size*batch_size)
        pi = utils.simplexCVX(batch_size,c);
        pi = torch.from_numpy(pi).float().cuda()
        #print pi[0,:]
    #assert(type(pi)!=Nonetype)
    loss = torch.sum(torch.mul(cost_matrix,pi))
    return loss,pi

def train(g_net):
    data = dset.CIFAR10(root="/home/lrh/dataset/cifar-10",train = False, download=True,transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ]))

    dataloader = torch.utils.data.DataLoader(data,batch_size=batch_size,shuffle=True,drop_last=True)
    g_optimizer = optim.Adam(g_net.parameters(),lr=lr,betas = beta)

    for i,data in enumerate(dataloader,0):
        #     #with shape [batch_size,3,32,32]
        real_x = data[0]
    for epoch in range(epoch_num):
        z = torch.randn(real_x.size(0),nz);
        #to GPU
        if cuda:
            real_x = real_x.cuda()
            z = z.cuda()

        g_optimizer.zero_grad()
        isopt = True
        pi = None
        for j in range(g_steps):
            fake_x = g_net(z)
            #[batch_size,3,32,32]
            loss,pi = loss_fn(real_x,fake_x,isopt,pi)
            loss.backward()
            g_optimizer.step()
            isopt = False


        fake_x = g_net(z)
        vutils.save_image(fake_x.cpu().detach(),'%s/real_samples_epoch_%03d.png'
        % (result_directory,epoch),normalize=True)
        if epoch%50==0:
            torch.save(g_net.state_dict(),'%s/gnet_%03d.pkl' %(result_directory,epoch));
            print "epoch is:[{}|{}],index is:[{}|{}],g_loss:{}".\
                format(epoch,epoch_num,\
                i,len(dataloader),loss);

if __name__=="__main__":

    g_net = wgan_generator()
    g_net.apply(weights_init)
    if cuda:
        g_net.cuda()
    #set moudle.istraing=True
    #net.train()
    train(g_net)


## mnist 生成

结论1：

该模型可以学习到少量样本的分布，

比如只取一个batch用作训练样本。

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torch.optim as optim
import torchvision.utils as vutils
import numpy as np
from scipy.optimize import linprog
import torch.nn.functional as F
import utils

nz = 100
g_steps = 5
result_directory = "result_cifar_1009_direct_0.05LAMBDA"
batch_size = 64
bn = True
lr = 3e-4
beta = (0.5,0.999)
epoch_num = 1000
cuda = True


a1 = np.zeros([batch_size*batch_size,batch_size]);
a2 = np.zeros([batch_size*batch_size,batch_size])
for i in range(batch_size):
    a1[batch_size*i:batch_size*(i+1),i] = 1
    a2[batch_size*i:batch_size*(i+1),:] = np.eye(batch_size,batch_size)
A = np.concatenate((a1,a2),axis=1)
A = A.T
b = np.ones([batch_size*2])/batch_size


class wgan_generator(nn.Module):
    def __init__(self):
        super(wgan_generator,self).__init__()
        #input: [batch_size,100]
        self.fc1 = nn.utils.weight_norm(nn.Linear(nz,2*4*4*1024))

        self.conv1 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(1024,2*512,kernel_size=5,padding=2,stride=1))
        )

        self.conv2 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(512,2*256,kernel_size=5,padding=1,stride=1))
        )

        self.conv3 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(256,2*128,kernel_size=5,padding=2,stride=1))
        )

        self.conv4 = nn.utils.weight_norm(nn.Conv2d(128,1,kernel_size=5,padding=2,stride=1))



    def forward(self,x):

        x = self.fc1(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l) # gated linear unit, one of Alec's tricks

        x = x.view(-1,1024,4,4)
        #[4,4]
        x = self.conv1(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[8,8]
        x = self.conv2(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[14,14]
        x = self.conv3(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[28,28]
        x = self.conv4(x)

        x = F.tanh(x)
        return x;

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

def loss_fn(real_x,fake_x,isopt=True,pi=None):
    batch_size = real_x.size(0)
    cost_matrix = torch.zeros([batch_size,batch_size]).cuda()
    dist_matrix = np.zeros([batch_size,batch_size])
    for i in range(batch_size):
        diff = real_x - fake_x[i]
        diff = diff.view(diff.size(0),-1)
        #||x_i - y_j||_2
        #[batch_size,1]
        diff_norm = diff.norm(2 , dim = 1)
        diff_norm_1 = diff.norm(1 , dim = 1)
        cost_matrix[:,i] = diff_norm + 0.05 * diff_norm_1
        #min_value,min_pos = torch.min(diff_norm,dim=0)
        #print min_pos
        #dist_matrix[min_pos,i] = 1.0/batch_size
    #[batch_size,batch_size]
    if isopt:
        c = cost_matrix.cpu().detach().numpy().reshape(batch_size*batch_size)
        pi = utils.simplexCVX(batch_size,c);
        pi = torch.from_numpy(pi).float().cuda()
        #print pi[0,:]
    #assert(type(pi)!=Nonetype)
    loss = torch.sum(torch.mul(cost_matrix,pi))
    return loss,pi

def train(g_net):
    data = dset.MNIST(root="/home/lrh/dataset/mnist",train = False, download=False,transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ]))

    dataloader = torch.utils.data.DataLoader(data,batch_size=batch_size,shuffle=True,drop_last=True)
    g_optimizer = optim.Adam(g_net.parameters(),lr=lr,betas = beta)

    for i,data in enumerate(dataloader,0):
        #     #with shape [batch_size,3,32,32]
        real_x = data[0]
    for epoch in range(epoch_num):
        z = torch.randn(real_x.size(0),nz);
        #to GPU
        if cuda:
            real_x = real_x.cuda()
            z = z.cuda()

        g_optimizer.zero_grad()
    #
        isopt = True
        pi = None
        for j in range(g_steps):
            fake_x = g_net(z)
            #[batch_size,3,32,32]
            loss,pi = loss_fn(real_x,fake_x,isopt,pi)
            loss.backward()
            g_optimizer.step()
            isopt = False
            
        fake_x = g_net(z)
        vutils.save_image(fake_x.cpu().detach(),'%s/real_samples_epoch_%03d.png'
        % (result_directory,epoch),normalize=True)
        if epoch%50==0:
            torch.save(g_net.state_dict(),'%s/gnet_%03d.pkl' %(result_directory,epoch));
            print "epoch is:[{}|{}],index is:[{}|{}],g_loss:{}".\
                format(epoch,epoch_num,\
                i,len(dataloader),loss);

    
if __name__=="__main__":

    g_net = wgan_generator()
    g_net.apply(weights_init)
    if cuda:
        g_net.cuda()
    #set moudle.istraing=True
    #net.train()
    train(g_net)


## mnist 

test_dataset

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torch.optim as optim
import torchvision.utils as vutils
import numpy as np
from scipy.optimize import linprog
import torch.nn.functional as F
import utils

nz = 100
g_steps = 5
result_directory = "result_mnist_1011_test_data_0.05LAMBDA"
batch_size = 64
bn = True
lr = 3e-4
beta = (0.5,0.999)
epoch_num = 1000
cuda = True


a1 = np.zeros([batch_size*batch_size,batch_size]);
a2 = np.zeros([batch_size*batch_size,batch_size])
for i in range(batch_size):
    a1[batch_size*i:batch_size*(i+1),i] = 1
    a2[batch_size*i:batch_size*(i+1),:] = np.eye(batch_size,batch_size)
A = np.concatenate((a1,a2),axis=1)
A = A.T
b = np.ones([batch_size*2])/batch_size


class wgan_generator(nn.Module):
    def __init__(self):
        super(wgan_generator,self).__init__()
        #input: [batch_size,100]
        self.fc1 = nn.utils.weight_norm(nn.Linear(nz,2*4*4*1024))

        self.conv1 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(1024,2*512,kernel_size=5,padding=2,stride=1))
        )

        self.conv2 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(512,2*256,kernel_size=5,padding=1,stride=1))
        )

        self.conv3 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.utils.weight_norm(nn.Conv2d(256,2*128,kernel_size=5,padding=2,stride=1))
        )

        self.conv4 = nn.utils.weight_norm(nn.Conv2d(128,1,kernel_size=5,padding=2,stride=1))



    def forward(self,x):

        x = self.fc1(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l) # gated linear unit, one of Alec's tricks

        x = x.view(-1,1024,4,4)
        #[4,4]
        x = self.conv1(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[8,8]
        x = self.conv2(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[14,14]
        x = self.conv3(x)
        x,l = torch.chunk(x,2,dim=1)
        x = x * F.sigmoid(l)
        #[28,28]
        x = self.conv4(x)

        x = F.tanh(x)
        return x;

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

def loss_fn(real_x,fake_x,isopt=True,pi=None):
    batch_size = real_x.size(0)
    cost_matrix = torch.zeros([batch_size,batch_size]).cuda()
    dist_matrix = np.zeros([batch_size,batch_size])
    for i in range(batch_size):
        diff = real_x - fake_x[i]
        diff = diff.view(diff.size(0),-1)
        #||x_i - y_j||_2
        #[batch_size,1]
        diff_norm = diff.norm(2 , dim = 1)
        diff_norm_1 = diff.norm(1 , dim = 1)
        cost_matrix[:,i] = diff_norm + 0.05 * diff_norm_1
        #min_value,min_pos = torch.min(diff_norm,dim=0)
        #print min_pos
        #dist_matrix[min_pos,i] = 1.0/batch_size
    #[batch_size,batch_size]
    if isopt:
        c = cost_matrix.cpu().detach().numpy().reshape(batch_size*batch_size)
        pi = utils.simplexCVX(batch_size,c);
        pi = torch.from_numpy(pi).float().cuda()
        #print pi[0,:]
    #assert(type(pi)!=Nonetype)
    loss = torch.sum(torch.mul(cost_matrix,pi))
    return loss,pi

def train(g_net):
    data = dset.MNIST(root="/home/lrh/dataset/mnist",train = False, download=False,transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ]))

    dataloader = torch.utils.data.DataLoader(data,batch_size=batch_size,shuffle=True,drop_last=True)
    g_optimizer = optim.Adam(g_net.parameters(),lr=lr,betas = beta)

    
    for epoch in range(epoch_num):
        for i,data in enumerate(dataloader,0):
        #with shape [batch_size,3,32,32]
            real_x = data[0]
            z = torch.randn(real_x.size(0),nz);
            #to GPU
            if cuda:
                real_x = real_x.cuda()
                z = z.cuda()

            g_optimizer.zero_grad()
        #
            isopt = True
            pi = None
            for j in range(g_steps):
                fake_x = g_net(z)
                #[batch_size,3,32,32]
                loss,pi = loss_fn(real_x,fake_x,isopt,pi)
                loss.backward()
                g_optimizer.step()
                isopt = False

        fake_x = g_net(z)
        vutils.save_image(fake_x.cpu().detach(),'%s/real_samples_epoch_%03d.png'
        % (result_directory,epoch),normalize=True)
        if epoch%50==0:
            torch.save(g_net.state_dict(),'%s/gnet_%03d.pkl' %(result_directory,epoch));
            print "epoch is:[{}|{}],index is:[{}|{}],g_loss:{}".\
                format(epoch,epoch_num,\
                i,len(dataloader),loss);

    
if __name__=="__main__":

    g_net = wgan_generator()
    g_net.apply(weights_init)
    if cuda:
        g_net.cuda()
    #set moudle.istraing=True
    #net.train()
    train(g_net)


In [ ]:
import torch
a = torch.Tensor([[1,2,4,5]])
print a.transpose(1,0)

## 在特征空间匹配

1.autoencoder

2.inception-v3 classify